<a href="https://colab.research.google.com/github/takatakamanbou/Data/blob/2023/ex14notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data2023 ex14notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/Data-logo14.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?Data/2023



## 準備

以下，コードセルを上から順に実行していってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# Python 用の機械学習ライブラリ scikit-learn の線形回帰モデルおよびロジスティック回帰モデル
from sklearn.linear_model import LinearRegression, LogisticRegression

# ゴリゴリくん
dfGori = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/ex08gorigori.csv', header=0)

# 模試入試
dfExam = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/ex14exam.csv', header=0)

# 小テスト + 睡眠時間 vs 期末テスト
dfScore = pd.read_csv('https://github.com/ghmagazine/python_stat_sample/raw/master/data/ch12_scores_reg.csv')
dfScore.drop(columns='通学方法', inplace=True)

----
## 「データ分析」から「多変量解析及び演習」・「機械学習I/II」へ
----

----
### 多変量解析？

「データ分析」で扱ったデータは，変数の数が1つまたは2つに限られていました．
「多変量解析及び演習」では，変数（説明変数）が複数あるようなデータ（多変量データ）を扱うデータ分析・統計手法である **多変量解析** について学びます．
多変量解析ができるようになれば，より多様なデータ／より複雑な問題に取り組むことができるようになります．



「多変量解析」の手法には，例えば次のようなものがあります．

- **重回帰分析**
- **主成分分析**
- **クラスター分析**／**クラスタリング**

以下では，重回帰分析について少し説明します．
それ以外のものはここでは説明しませんので，興味があれば書籍を探したりネットで調べたりしてみるとよいでしょう．また，以下のリンク先に，2023年度「多変量解析及び演習」のページがあります．講義資料は Colab Notebook として自由に閲覧できますので，よかったらどうぞ．
https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023



----
### 重回帰分析？

**重回帰分析** は，回帰分析を説明変数が2つ以上ある場合に拡張したものです（対比させるときは，説明変数1つの回帰分析を **単回帰**（分析） と呼ぶことがあります）．


#### 「データ分析」で学んだ回帰分析


「データ分析」の授業で扱った回帰分析では，与えられるデータは

$$
(x_n, y_n)\quad (n = 1, 2, \dots , N)
$$

という形で，説明変数（$x$）も被説明変数（$y$）も一つだけでした．回帰分析の目的は，このようなデータにうまくあてはまる直線

$$y = ax+b$$

を見つける（パラメータ $a, b$ を決める），というものでした．この授業では，気温を説明変数，アイス売上数を被説明変数として，気温からアイス売上数を予測する，といった問題などを例として考えました．



In [ ]:
# ゴリゴリくん
dfGori.head()  # 最初の5件だけ表示

In [ ]:
# データ
X = dfGori['気温'].values  # 説明変数
Y = dfGori['アイス売上数'].values  # 被説明変数

# 最小二乗法による直線のあてはめ
lr = LinearRegression().fit(X.reshape(-1, 1), Y)
a, b = lr.coef_[0], lr.intercept_

# 予測
Yest = a * X + b

# グラフを描く
Xr = np.array([0, 40])
fig, ax = plt.subplots(1, facecolor='white', figsize=(9, 6))
ax.set_xlabel('Temparature')
ax.set_ylabel('number of GoriGori')
ax.scatter(X, Y)
ax.plot(Xr, a*Xr + b, color='red')
ax.set_xlim(0, 40)
ax.set_ylim(-5, 125)
plt.show()

# 得られたパラメータを表示
print(f'a = {a:.3f}, b = {b:.3f}')

#### 重回帰分析の問題の例

では，説明変数が2つ以上ある場合とはどういうものか，実際のデータの例をあげてみます．

In [ ]:
dfScore

ここで扱うデータは，

「Pythonで理解する統計解析の基礎」 谷合廣紀，辻 真吾，技術評論社，2018.

に掲載されているものです（以下の GitHub サイトで公開されています． https://github.com/ghmagazine/python_stat_sample ）．
ある大学のある授業を受講している20人について，小テストの点数，期末テストの点数，睡眠時間，3つの情報を集めたものとなっています（実際には通学方法という項目もありますがここでは省略してます）．

このようなデータが与えられた場合，「小テスト」の点数を $x_1$，「睡眠時間」を $x_2$，「期末テスト」の点数を $y$ とおいて，

$$
y = w_0 + w_1x_1 + w_2x_2
$$

という式を考え，データに当てはまるパラメータ $w_0, w_1, w_2$ を決めることができれば，「小テスト」，「睡眠時間」の値と「期末テスト」の値の間にどんな関係があるのかを知ることができます．
重回帰分析は，このようなデータ分析を行うための手法です．「小テスト」と「睡眠時間」が説明変数で，「期末テスト」が被説明変数です．この例では説明変数の数は2つですが，3つ以上の説明変数を考えることも可能です．

#### 実際に重回帰分析してみよう

上記データの重回帰分析を実際にやってみましょう．

In [ ]:
# データの準備
X = dfScore.drop(columns='期末テスト').values # 説明変数
Y = dfScore['期末テスト'].values  # 被説明変数

# 最小二乗法による平面のあてはめ
lr = LinearRegression().fit(X, Y)
w0 = lr.intercept_
w  = lr.coef_

# 得られたパラメータを表示
print(f'w0 = {w0:.2f},   w1 = {w[0]:.2f},   w2 = {w[1]:.2f}')

上記の結果は，「小テスト」と「睡眠時間」から「期末テスト」の値を予測する式が

$$
y = -1.87 + 6.43x_1 + 4.19x_2
$$

と求まることを意味しています．小テストの点数 ($x_1$) が高いほど，睡眠時間 ($x_2$) が長いほど，期末テストの点数 ($y$) は高くなるようです．
これを使って実際にデータから「期末テスト」の点数を予測してみると...

In [ ]:
# 予測値の計算
Y_est = lr.predict(X)
for n in range(len(X)):
    print(f'{n:2d} 説明変数の値: {X[n]}  被説明変数の予測値: {Y_est[n]:.1f}  真の値: {Y[n]:d}')

これだけでは予測がうまく言ってるかどうか判断が難しい（注）ですが，それっぽい値を出しているようです

<font size="-1">注） 単回帰の場合に **決定係数** を考えたように，重回帰分析でも当てはまりの良さや結果の信頼性を評価する方法があります．</font>

単回帰分析の場合，説明変数を $x$，被説明変数を $y$ として，$(x,y)$ 平面にデータに当てはまる直線を引くことになっていましたが，上記の重回帰分析の場合，$(x_1, x_2, y)$ 空間に平面を当てはめることになっています．図に描くとこんなんなります．

In [ ]:
# 平面当てはめの可視化
x = np.linspace(0, 10, num=11)
y = np.linspace(0, 10, num=11)
xx, yy = np.meshgrid(x, y)
zz = w0 + w[0] * xx + w[1] * yy

elevation, azimuth = 40, -30

fig = plt.figure(facecolor='white', figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d', xlabel='$x_2$', ylabel='$x_1$', zlabel='$y$')
ax.scatter(X[:, 0], X[:, 1], Y)
ax.plot_wireframe(xx, yy, zz, color='red')
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.view_init(elevation, azimuth)
plt.show()

この図の赤線で描かれた平面は

$$
y = -1.87 + 6.43x_1 + 4.19x_2
$$

を表しています．

#### 重回帰分析の定式化

説明変数の数を $D$，データの数（サンプルサイズ）を $N$とすると，重回帰分析で扱うデータは次のように表せます．

$$
(x_{n, 1}, x_{n, 2}, \dots, x_{n, D}, y_n)\quad (n = 1, 2, \dots , N)
$$

$n$ はデータの番号，$x_{n, 1}$ から $x_{n, D}$ までが $n$ 番目のデータの説明変数の値，$y_n$ が被説明変数の値を表わします． 上記の例では $D=2$ および $N=20$ でした．

このデータに当てはめる式は，次のようなものです．

$$
y = w_0 + w_1x_1 + w_2x_2 + \dots + w_Dx_D
$$

単回帰の式 $y = ax+b$ が2次元平面上の直線の式だったのに対して，この式は，データが広がる $(D+1)$ 次元空間中の平面（$D$次元超平面）を表わします．
平面の位置や傾きを決めるパラメータは $w_0$ から $w_D$ までの $(D+1)$ 個あります．
これらのパラメータは，単回帰の場合と同様に，最小二乗法によって求めることができます（注）．

<font size="-1">注） 単回帰の場合よりも複雑にはなりますが，微積分と線形代数を駆使してパラメータの連立方程式である「正規方程式」を導出することができます．
その辺のことは「多変量解析及び演習」で登場予定です．
</font>


----
### 重回帰分析のさらに先へ

重回帰分析のさらに先にも，いろいろ面白いものがあります．
「機械学習」で学ぶことになるだろう内容ですが，一部簡単に紹介します．

#### 多項式の当てはめ and more

重回帰分析の式をちょっと変形すると，単回帰で扱ったデータ
$$
(x_n, y_n)\quad (n = 1, 2, \dots , N)
$$
に対して，直線ではなく $D$ 次多項式

$$ y = w_0 + w_1x + w_2x^2 + \dots +
w_Dx^D
$$

を当てはめることができます．

「ゴリゴリくん」でやってみましょう．以下のスライダーを動かして多項式の次数 $D$ を 1 から大きくしていってみよう．
$D$ を大きくしていくと，大変なことになります...


In [ ]:
#@title あてはめる多項式の次数 {run: "auto"}
D = 5 #@param {type:"slider", min:1, max:20}

# ゴリゴリくん
X, Y = dfGori['気温'], dfGori['アイス売上数']
N = Y.shape[0]

# データの行列 XX をつくる
XX = np.empty((N, D+1))
XX[:, 0] = 1
for d in range(1, D+1):
    XX[:, d] = X**d

# 最小二乗法で多項式を求める
lr = LinearRegression().fit(XX, Y)
f_est = np.poly1d(np.hstack((lr.intercept_, lr.coef_[1:]))[::-1])

# グラフを描く
xr = np.linspace(0, 40, 100)
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlabel('Temparature')
ax.set_ylabel('number of GoriGori')
ax.scatter(X, Y, color="red")
ax.plot(xr, f_est(xr), '-', label=f"D = {D}")
ax.set_xlim(0, 40)
ax.set_ylim(-5, 125)
ax.legend()
plt.show()

# 得られた多項式を表示
print(f_est)

この問題のように，与えられたデータに複雑な関数を当てはめる問題（**関数近似** と言うことがあります）は，科学技術の様々な問題に現れる重要なものです．
関数近似問題への取り組み方（アプローチ）にも様々なものがありますので，数理・情報科学課程のあちこちの授業で出会うことになるかもしれません（微積分で学んだ「テイラー展開」も，その特殊な例と言えます）．
いわゆる人工知能の基礎となる話でもありますので，「機械学習I/II」でも取り上げます．


#### ロジスティック回帰

これまでの回帰分析の話では，被説明変数はすべて量的なものでした（注）．
**ロジスティック回帰** は，被説明変数が質的なものでかつ2種類の値しかとらないようなデータを分析するための手法です．

<font size="-1">注）  説明変数の方については，質的な変数が混じっていたとしても，ダミー変数に変換してやることで，量的な変数とみなして普通に回帰分析することができます．</font>

例えばこんなん...．ある模擬試験を受験したのちある大学の入試を受験したひと300人分の，模試の2科目の点数と，大学入試に合格したか否か（1なら合格），のデータです．

In [ ]:
# 模試入試データ
dfExam

模試の点数から合否を予測する式を作ることができれば，このデータに含まれない受験生の合否を予測できるかもしれません．このような問題を扱うのが，ロジスティック回帰です．

とりあえず，説明変数（模試2科目の点数）の散布図を描いてみるとこんなんなります．

In [ ]:
# データ
X = dfExam[['模試数学', '模試英語']].values # 説明変数
Y = dfExam['入試合否'].values # 被説明変数

fig, ax = plt.subplots(1, 2, facecolor='white', figsize=(12, 6))
# 普通に散布図
ax[0].scatter(X[:, 0], X[:, 1])
ax[0].set_xlim(0, 100)
ax[0].set_ylim(0, 100)
ax[0].set_xlabel('Math')
ax[0].set_ylabel('English')
# 入試合否で色分け
X0 = X[Y == 0, :]
X1 = X[Y == 1, :]
ax[1].scatter(X0[:, 0], X0[:, 1])
ax[1].scatter(X1[:, 0], X1[:, 1])
ax[1].set_xlim(0, 100)
ax[1].set_ylim(0, 100)
ax[1].set_xlabel('Math')
ax[1].set_ylabel('English')
plt.show()

右の方は，合格したひとをオレンジで，不合格だったひとを青で表しています．さらに，入試合否も使って3次元の散布図を描くと，こんなことになります．

In [ ]:
# 模試の点数と入試合否の3次元の散布図
fig = plt.figure(facecolor='white', figsize=(10, 7.5))
ax = fig.add_subplot(111, projection='3d', xlabel='Math', ylabel='English', zlabel='result')
X0 = X[Y == 0, :]
X1 = X[Y == 1, :]
ax.scatter(X0[:, 0], X0[:, 1], Y[Y == 0])
ax.scatter(X1[:, 0], X1[:, 1], Y[Y == 1])
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_zlim(0, 1)
elevation = 70
azimuth = 240
ax.view_init(elevation, azimuth)
plt.show()

以下，ロジスティック回帰の式などの説明は省略して，結果を図示するだけにします．

In [ ]:
# データ
X = dfExam[['模試数学', '模試英語']].values # 説明変数
Y = dfExam['入試合否'].values # 被説明変数

# ロジスティック回帰
lr = LogisticRegression(C=1e6).fit(X, Y)
w0 = lr.intercept_[0]
w = lr.coef_[0]

# パラメータを表示
print(w0, w)

In [ ]:
# 予測値
x0 = np.arange(0, 100, 5)
x1 = np.arange(0, 100, 5)
xx0, xx1 = np.meshgrid(x0, x1)
XX = np.vstack((xx0.reshape(-1), xx1.reshape(-1))).T
yy = lr.predict_proba(XX)[:, 1].reshape(xx0.shape)

# 可視化
fig = plt.figure(facecolor='white', figsize=(10, 7.5))
ax = fig.add_subplot(111, projection='3d', xlabel='Math', ylabel='English', zlabel='result')
X0 = X[Y == 0, :]
X1 = X[Y == 1, :]
ax.scatter(X0[:, 0], X0[:, 1], Y[Y == 0])
ax.scatter(X1[:, 0], X1[:, 1], Y[Y == 1])
ax.plot_wireframe(xx0, xx1, yy, color='green')
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_zlim(0, 1)
elevation = 70
azimuth = 240
ax.view_init(elevation, azimuth)
plt.show()

上記の緑色は，ロジスティック回帰によって得られた予測式が表す曲面です．この式によって得られる予測値は，「合格する確率」と解釈することができます．

以下は，予測値が 0.5 に等しくなるような説明変数の値の集合（緑色の直線）を描いたものです．

In [ ]:
fig, ax = plt.subplots(1, facecolor='white', figsize=(6, 6))
X0 = X[Y == 0, :]
X1 = X[Y == 1, :]
ax.scatter(X0[:, 0], X0[:, 1])
ax.scatter(X1[:, 0], X1[:, 1])
xr = np.linspace(0, 100)
ax.plot(xr, -(w[0]*xr + w0)/w[1], color='green')
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_xlabel('Math')
ax.set_ylabel('English')
plt.show()

大雑把な考察ですが，この直線の傾きから，模試英語の点数よりも模試数学の点数の大小の方が入試合否に強く影響していることがうかがえます．

ここでは説明しませんが，このロジスティック回帰を発展させて，ずっとずっと複雑な仕組みを考えると，文字認識，音声認識，画像認識といった問題も扱えるようになります．
そういう話は，「機械学習I/II」の授業に出てくるでしょう．